First of all, make sure your environment has installed the latest version of [🤗 Optimum Graphcore](https://github.com/huggingface/optimum-graphcore).

In [ ]:
%pip install git+https://github.com/huggingface/optimum-graphcore.git

Also make sure all the packages required for this notebook are installed.

In [ ]:
%pip install scikit-learn;
%pip install datasets
%pip install evaluate
%pip install tokenizers
%pip install matplotlib
%pip install scipy
%pip install --force-reinstall huggingface_hub==0.11.1;

Let's start by importing and printing out the versions of `Transformers` and `Optimum Graphcore`:

In [1]:
import transformers
import optimum.graphcore

print(transformers.__version__)
print(optimum.graphcore.__version__)

4.25.1
0.6.0.dev0


At the end of this notebook, to be able to share your model with the community and easily access it through HuggingFace, there are some short set-up steps you must follow to enable uploading your checkpoint to the HuggingFace Hub.

First you have to store your authentication token from the Hugging Face website ([sign up here](https://huggingface.co/join) if you haven't already!) then execute the following cell and input your username and password:

In [2]:
from huggingface_hub import notebook_login

notebook_login()

Git-lfs must also be installed to enable large file storage when pushing to the hub:

In [ ]:
!apt install git-lfs

# Faster multi-label text classification with PackedBERT

This notebook builds on the process of fine-tuning BERT on a [text classification task](text_classification.ipynb) showing how to implement packing for BERT for a new sequence classification task - multi-label classification. [Packing](https://www.graphcore.ai/posts/introducing-packed-bert-for-2x-faster-training-in-natural-language-processing), an optimisation method originally used for 2x faster BERT pre-training, which can now also provide massive throughput increases for **fine-tuning** and **batched inference**! 

**So, what *is* packing?** The basic idea of 'packing' a dataset is to utilise the requirement for constant-shaped inputs into a model. Instead of padding it with empty, unused space, we can recycle this unused space and fill it with more inputs! The architecture of transformer models like BERT supports this, and lets us optimally use this space to process multiple sequences within one input.

**And here is why you might want to use it:** Having a single input contain multiple sequences leads to multiple sequences being processed in parallel in a single pass within a single iteration inside a batch, increasing the 'effective' batch size of the model by a considerable factor in many cases, and most importantly, increasing model throughput for training and batched inference significantly.

This notebook outlines how to easily enable packing for BERT when performing fine-tuning/inference on a text-classification task in 🤗 Optimum, resulting in an impressive 5-6x faster training and inference run-time on the `go-emotions` dataset. 

You can read more about packing in the original [paper](https://arxiv.org/abs/2107.02027).

The [GoEmotions](https://ai.googleblog.com/2021/10/goemotions-dataset-for-fine-grained.html) dataset will be fine-tuned using packing. . This notebook outlines how to easily enable packing for BERT when performing fine-tuning/inference on a text-classification task in 🤗 Optimum, resulting in an impressive 5-9x faster training and inference run-time for the dataset. 



![GoEmotions dataset (Source: GoogleBlog)](../images/go_emotions.png)

The dataset consists of 58k comments labelled for 27 different emotion categories (or Neutral). This dataset is used for multi-label, multi-class classification. The dataset format and categories can be viewed on the [Huggingface Hub](https://huggingface.co/datasets/go_emotions).

Let's initialise our training configurations. 

In this notebook, we are using both data parallelism and pipeline parallelism (see this [tutorial](https://github.com/graphcore/tutorials/tree/master/tutorials/pytorch/tut2_efficient_data_loading) for more). Therefore the global batch size, which is the actual number of samples used for the weight update, is determined using three factors:

    global batch size = micro_batch_size * gradient accumulation steps * replication factor

and replication factor is determined by pod_type, which will be used as a key to select the replication factor from a dictionary defined in the IPU config file. For example, the dictionary in the IPU config file Graphcore/roberta-base-ipu looks like this:

    "replication_factor": {"pod4": 1, "pod8": 2, "pod16": 4, "pod32": 8, "pod64": 16, "default": 1}

Depending on you model and the pod machine you are using, you might need to adjust these three batch-size-related arguments.

By default this notebook is configured to run on 4 IPUs.

Finally, max_seq_length is the length we are going to pad the sentences to, so it should not be larger than the maximum length of the model. Set those seven parameters, then the rest of the notebook should run smoothly:

Given the small size of the sequences in go-emotions, we can reduce the model maximum input size to `max_seq_length = 256`.


In [3]:
model_checkpoint = "bert-base-uncased" # Default uncased pre-trained BERT checkpoint
ipu_config_name = "Graphcore/bert-base-uncased" # Default Graphcore IPU config initialisation for pre-trained BERT
max_seq_length = 256 # The maximum sequence length allowed for sequences in the model.
micro_batch_size = 2 
gradient_accumulation_steps = 39
model_task = 'go_emotions'
num_labels = 28

Gradients are not calculated during validation, so gradient accumulation is not applicable, and the global batch size for validation can be defined separately as:

```
global_validation_batch_size=device_iterations*replication_factor*max_seq_per_pack
```


Values for machine size and cache directories can be configured through environment variables or directly in the notebook:

In [4]:
import os

pod_type = os.getenv("GRAPHCORE_POD_TYPE", "pod4")
executable_cache_dir = os.getenv("POPLAR_EXECUTABLE_CACHE_DIR", "./exe_cache/") + "/packed_bert_mlseqcls/"

## Loading the dataset

We will use the [🤗 Datasets](https://github.com/huggingface/datasets) library to download the data and get the metric we need to use for evaluation (to compare our model to the benchmark). This can be easily done with the functions `load_dataset` and `load_metric`.  

In [5]:
from datasets import load_dataset
import evaluate

In [6]:
dataset = load_dataset(model_task)
metric = evaluate.load("roc_auc", "multilabel")

No config specified, defaulting to: go_emotions/simplified
Found cached dataset go_emotions (/home/arsalanu/.cache/huggingface/datasets/go_emotions/simplified/0.0.0/2637cfdd4e64d30249c3ed2150fa2b9d279766bfcd6a809b9f085c61a90d776d)


  0%|          | 0/3 [00:00<?, ?it/s]

The `dataset` object itself is [`DatasetDict`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasetdict), which contains one key for the training, validation and test set (with more keys for the mismatched validation and test set in the special case of `mnli`).

In [7]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'labels', 'id'],
        num_rows: 43410
    })
    validation: Dataset({
        features: ['text', 'labels', 'id'],
        num_rows: 5426
    })
    test: Dataset({
        features: ['text', 'labels', 'id'],
        num_rows: 5427
    })
})

To access an actual element, you need to select a split first, then give an index:

In [8]:
dataset["train"][0]

{'text': "My favourite food is anything I didn't have to cook myself.",
 'labels': [27],
 'id': 'eebbqej'}

To get a sense of what the data looks like, the following function will show some examples picked randomly in the dataset.

In [9]:
import datasets
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [10]:
show_random_elements(dataset["train"])

,text,labels,id
0,This feels like a reference to [NAME] from Homestar Runner but I'm not completely sure.,[27],eez4wec
1,“So even if vaccination did cause autism...WHICH IT FUCKING DOESN’T!” Pure genius.,[0],ef2q8cx
2,"> Woo that sadness out Welp, never gonna be able to have a night out on the town again without this coming up",[25],edb9yy1
3,Can’t wait to go!,[13],ef43gz5
4,"I dated one for about a month, and just one or two dates with a couple other guys.",[27],ed8x9f9
5,Honey mustard.,[27],edx75wl
6,That seems pretty rough. If I ask Google for directions from white center to Fiona it’s only an hour by bicycle even.,[27],edowgt0
7,That seems crazy that you can't hit higher frame rates. I can hit 80fps consistent with a 1080. Something weird is going on here.,[9],ed53zzb
8,DO NOT SAY ANYONE OTHER THAN [NAME] IS BETTER THAN [NAME] OR THE PITCHFORKS ARE OUT,[27],edopwpl
9,Shame our government can't progress as fast as the infrastructure.,"[9, 12]",edavq34


The metric is an instance of [`datasets.Metric`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasets.Metric):

In [11]:
metric

EvaluationModule(name: "rocauc", module_type: "metric", features: {'references': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None), 'prediction_scores': Sequence(feature=Value(dtype='float32', id=None), length=-1, id=None)}, usage: """
Args:
- references (array-like of shape (n_samples,) or (n_samples, n_classes)): Ground truth labels. Expects different input based on use case:
    - binary: expects an array-like of shape (n_samples,)
    - multiclass: expects an array-like of shape (n_samples,)
    - multilabel: expects an array-like of shape (n_samples, n_classes)
- prediction_scores (array-like of shape (n_samples,) or (n_samples, n_classes)): Model predictions. Expects different inputs based on use case:
    - binary: expects an array-like of shape (n_samples,)
    - multiclass: expects an array-like of shape (n_samples, n_classes)
    - multilabel: expects an array-like of shape (n_samples, n_classes)
- average (`str`): Type of average, and is ignored in the bina

You can call its `compute` method with your predictions and labels directly and it will return a dictionary with the metric(s) value:

## Preprocessing the data

Before we can feed the texts to our model, we need to preprocess them. This is done by a 🤗 Transformers `Tokenizer` which will (as the name indicates) tokenize the inputs (including converting the tokens to their corresponding IDs in the pretrained vocabulary) and put it in a format the model expects, as well as generate the other inputs that model requires.

To do all of this, we instantiate our tokenizer with the `AutoTokenizer.from_pretrained` method, which will ensure:

- we get a tokenizer that corresponds to the model architecture we want to use,
- we download the vocabulary used when pretraining this specific checkpoint.

That vocabulary will be cached, so it's not downloaded again the next time we run the cell.

In [ ]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

We pass along `use_fast=True` to the call above to use one of the fast tokenizers (backed by Rust) from the 🤗 Tokenizers library. Those fast tokenizers are available for almost all models, but if you got an error with the previous call, remove that argument.

You can directly call this tokenizer on one sentence or a pair of sentences:

In [ ]:
tokenizer("Hello, this one sentence!", "And this sentence goes with it.")

Depending on the model you selected, you will see different keys in the dictionary returned by the cell above. They don't matter much for what we're doing here (just know they are required by the model we will instantiate later), you can learn more about them in [this tutorial](https://huggingface.co/transformers/preprocessing.html) if you're interested.

To preprocess our dataset, we will need the names of the columns containing the sentence(s). In this case, the column is called `'text'` and it is indexed as such in the tokenization function.

We can then write the function that will preprocess our samples. We just feed them to the `tokenizer` with the three arguments.`truncation=True` will ensure that an input longer than maximum length will be truncated to the maximum length. `max_length=max_seq_length` sets the maximum length of a sequence.

**Note that since we use packing later, we don't set any padding in the tokenizer.**

In [ ]:
# no padding for packing
def preprocess_function(examples):
    return tokenizer(examples['text'], truncation=True, max_length=max_seq_length)

For multi-label classification, we also need to convert our labels from integer values indicating a category to an N-hot binary format (where N is the maximum number of labels. This makes sure we have constant sized labels, and all of our labels (one input can have multiple target labels) are present for training. The conversion looks something like this:

```python
unprocessed_labels = [3,21] # Where 3 and 21 are label categories
preprocessed_labels = id_to_N_hot([3,21])
preprocessed_labels = [0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0]
```


The following function processes one example and converts it to N-hot - the `.map()` functionality available in the `datasets` library allows the function to be applied easily to the entire dataset.

In [ ]:
import numpy as np

def id_to_N_hot(example):
    indexes = example['labels']
    label = np.zeros((num_labels,), dtype=int)
    for idx in indexes:
        label[idx] = 1
    example['labels'] = label
    return example

To apply this function on all the sentences (or pairs of sentences) in our dataset, we just use the `map` method of our `dataset` object we created earlier. This will apply the function on all the elements of all the splits in `dataset`, so our training, validation and testing data will be preprocessed in one single command.

In [ ]:
encoded_dataset = dataset.map(id_to_N_hot)
encoded_dataset = encoded_dataset.map(preprocess_function, batched=True)

len(encoded_dataset['validation'])

Even better, the results are automatically cached by the 🤗 Datasets library to avoid spending time on this step the next time you run your notebook. The 🤗 Datasets library is normally smart enough to detect when the function you pass to map has changed (and thus requires to not use the cache data). For instance, it will properly detect if you change the task in the first cell and rerun the notebook. 🤗 Datasets warns you when it uses cached files, you can pass `load_from_cache_file=False` in the call to `map` to not use the cached files and force the preprocessing to be applied again.

Note that we passed `batched=True` to encode the texts by batches together. This is to leverage the full benefit of the fast tokenizer we loaded earlier, which will use multi-threading to treat the texts in a batch concurrently.

##  Packing the dataset

To implement packing, we need to pack our dataset first. Each new element will be a "pack" containing at most `max_seq_per_pack` sequences.

In [ ]:
max_seq_per_pack = 6

The problem type for this task is multi_label_classification, this also needs to be defined for the packed model to work.

In [ ]:
problem_type = 'multi_label_classification'

### Packing algorithm

In order to pack efficiently, we will use a histogram-based algorithm: shortest-pack-first histogram packing (SPFHP) presented in the [blog post](https://www.graphcore.ai/posts/introducing-packed-bert-for-2x-faster-training-in-natural-language-processing) adapted from the [blog code](https://github.com/graphcore/tutorials/tree/master/blogs_code/packedBERT). The full process of packing the dataset consists of four steps:

1. Create a histogram of the sequence lengths of the dataset.
2. Generate the 'strategy' for the dataset using one of the state-of-the-art packing algorithms, which maps out the order and indices of the sequences that need to be packed together.
3. Use this strategy to create the actual dataset, concatenating the tokenized features together for each column in the dataset, including the labels.
4. Finally, pass these new columns into a custom PyTorch dataset, ready to be passed to the PopTorch dataloader!

These steps have been simplified through the easy-to-use `packing_utils` available in Graphcore Optimum. You can simply generate the packed dataset after the usual tokenization and preprocessing by passing all necessary packing configuration to the `PackedDatasetCreator` class, and generate the ready-to-use PyTorch dataset with `.create()`.

Within the function, there are some column names used by default. The expected default columns for text classification include:
* `input_ids`
* `attention_mask`
* `token_type_ids`
* `labels`

These should all be generated automatically when tokenizing any classification dataset for BERT. However, the labels key, as it is not encoded, may have a different name. For this dataset, the column key for the labels for this dataset is `labels`, we can pass this to the argument `custom_label_key`, so the class can find our labels. 

The `PackedDatasetCreator` requires different instantiations for different datasets, so it must be called separately for each of our dataset splits. We can set either `training`, `validation` or `inference` to `True` as needed.

In [ ]:
from utils.packing.dataset_creator import PackedDatasetCreator

train_data_packer = PackedDatasetCreator(
    tokenized_dataset = encoded_dataset['train'],
    max_sequence_length = max_seq_length,
    max_sequences_per_pack = max_seq_per_pack,
    training = True,
    num_labels = num_labels,
    problem_type = problem_type,
    algorithm = 'SPFHP',
    custom_label_key = 'labels'
)

val_data_packer = PackedDatasetCreator(
    tokenized_dataset = encoded_dataset['validation'],
    max_sequence_length = max_seq_length,
    max_sequences_per_pack = max_seq_per_pack,
    validation = True,
    num_labels = num_labels,
    problem_type = problem_type,
    algorithm = 'SPFHP',
    custom_label_key = 'labels'
)

This will create the strategy and initialise the necessary parameters for packing the dataset. We can see that the ideal speed-up we have achieved is approximately 5.7x the original dataset, which corresponds directly to the average packing factor: the average number of sequences within one pack.

The `PackedDatasetCreator` class also has some other features we do not use here for training, such as `pad_to_global_batch_size`, a feature useful for performing batched inference on a large samples when we do not want to lose any of the samples when creating data iterators using the `poptorch.Dataloader`, it applies 'vertical' padding to the dataset, adding filler rows to bring the dataset up to a value divisible by the global batch size, and allows for the largest possible batch sizes to be used without any loss of data.

You can also view the histogram generated in the first step of the packing process, to observe whether the distribution of sequence lengths in the dataset will benefit from packing - as a general rule, as long as the average length of the sequences in the dataset is 50% or less of the maximum sequence length, packing will offer at least a 2x throughput benefit, in other words: `throughput_increase ≈ max_seq_len/mean_seq_len`

Many datasets have distributions with much smaller average lengths, and will benefit much more. We can easily observe this distribution by retrieving and plotting the histogram from the data class:

In [ ]:
import matplotlib.pyplot as plt

train_histogram = train_data_packer.histogram

plt.hist(train_histogram, bins = [k for k in range(0,max_seq_length,10)]) 
plt.title("Sequence length histogram") 
plt.xlabel('Sequence lengths')
plt.ylabel('Frequency')
plt.show()

Now we need to create the actual packed dataset, this is the 3rd step of the packing process outlined above.

In this stage, we take the strategy for mapping the sequences by size into 'packs' that was generated by the packing algorithm, and use this to extract the sequences from the tokenized dataset, inserting them into packs for each column in the dataset. Any remaining space in a pack after the sequences have been concatenated is padded to bring all sequences up to the maximum sequence length.

Some key features unique to packed datasets are worth mentioning here:

- A specific `attention_mask` is generated: It contains a unique index for each sequence of the pack and `0` for the remaining padding tokens. This, essentially, tells the model where to "look" from the perspective of a single token, ignoring any encoded information (such as a different sequence) that is not relevant to that token.
    - Example of 3 sequences: `attention_mask = [1,1,1,1,1,1,2,2,2,2,2,3,3,3,3,3,0,...,0,1,2,3]`


- The [CLS] tokens of each sequence must be moved to the end of the pack.
    - For instance: `[CLS,a,b,c] + [CLS, d,e,f] + [CLS, g,h,i] -> [a,b,c,d,e,f,g,h,i,...,CLS,CLS,CLS]`
    

- The `position_ids` of a pack contain the concatenated `position_ids` of each sequences 
    - For instance given 3 sequences: `[0,1,2,3,4] + [0,1,2,3] + [0,1,2] -> [1,2,3,4,1,2,3,1,2,...,0,0,0]` (note: the CLS tokens position id '0' are also moved the end of the pack)
    
- `labels` and `token_type_ids` are also packed to correspond to the `input_ids` pack.


To create a dataloader-ready packed dataset, all you need to do is call the `create()` method:

In [ ]:
packed_train_dataset = train_data_packer.create()
packed_val_dataset = val_data_packer.create()

Let's visualize one sample of the new `packed_train_dataset`:

In [ ]:
packed_train_dataset[133]

## Fine-tuning the model

Now that our data is ready, we can download the pretrained model and fine-tune it.

### Implementing Packed BERT

A few model modifications are required to make packing work with BERT.
We extend the existing class `BertForSequenceClassification` to `PackedPipelinedBertForSequenceClassification` which incorporates the required changes to the pooler and the model output. The crux of these changes is to modify the generic sequence classification model to handle 'unpacking' multiple sequences in the output stage, treating them as a larger batch size for classification, as well as masking any padding created by packing.

First let's load a default BERT configuration using `AutoConfig`. The config includes a new parameter we must set, `max_sequences_per_pack`, this informs the model of the maximum number of sequences it will need to 'unpack' in the model output. It also allows us to clearly define the `num_labels` and `problem_type` for this model.

The problem type is essential to define here, as switching between methods used by different types of classification requires it within the custom model.

In [ ]:
from transformers import AutoConfig

config = AutoConfig.from_pretrained(model_checkpoint)
config.max_sequences_per_pack = max_seq_per_pack
config.num_labels = num_labels
config.problem_type = problem_type

Now we can instantiate the model class with the config, loading the weights from the model checkpoint.

In [ ]:
import torch
import numpy as np
torch.manual_seed(43)
np.random.seed(43)

from models.modeling_bert_packed import PackedPipelinedBertForSequenceClassification


model = PackedPipelinedBertForSequenceClassification(config).from_pretrained(
   model_checkpoint, config=config)

The warning is telling us we are throwing away some weights and randomly initializing some other. This is absolutely normal in this case, because we are removing the head used to pretrain the model on a masked language modeling objective and replacing it with a new head for which we don't have pretrained weights, so the library warns us we should fine-tune this model before using it for inference, which is exactly what we are going to do.

We can first test the model on CPU and observe that the output logits have the size `[batch_size x max_seq_per_pack, 2] = [12, 28]` with this notebook's default values, and the 28 labels for the dataset.

In [ ]:
# test the model on CPU
from transformers.data.data_collator import default_data_collator

loader = torch.utils.data.DataLoader(packed_train_dataset,
                             batch_size=1,
                             shuffle=True,
                             drop_last=True,
                             collate_fn=default_data_collator)
data = next(iter(loader))
labels = data['labels']

print('labels: ', labels.shape)
o = model(**data)
print('outputs (loss, logits): ', o[0], o[1].shape)

Now, let's prepare the model for IPU

First, we set the model in half precision:

In [ ]:
model.half()

For validation, we need to define a function to compute the metrics from the predictions, which will just use the `metric` we loaded earlier, preprocessing here involves a step to mask the labels and predictions we are not using, set to a `-100` value when creating the dataset, with a boolean mask. Then, the predictions are passed into a softmax function to determine the probabilities of each class, as this is a multi-label task. 

These predictions and labels are passed into the metric function to compute the accuracy during evaluation.

In [ ]:
model_name = model_checkpoint.split("/")[-1]
from scipy.special import softmax
from tqdm import tqdm
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    
    labels = labels.reshape(-1, labels.shape[-1])
    predictions = predictions.reshape(-1, predictions.shape[-1])
    
    # Remove the padding labels
    mask = (labels != -100)[:,0]
    
    labels = labels[mask,:]
    predictions = predictions[mask,:]
    pred_scores = softmax(predictions.astype("float32"), axis=1)    

    auc = metric.compute(
        prediction_scores=pred_scores, references=labels, multi_class="ovr"
    )["roc_auc"]

    return {"roc_auc": auc}

Next, we need to define the `IPUConfig`, which is a class that specifies attributes and configuration parameters to compile and put the model on the device. We initialize it with one config name or path, which we set earlier. Then we use it to set the mode attribute `model.ipu_config` 

In [ ]:
from optimum.graphcore import IPUConfig, IPUTrainer, IPUTrainingArguments

ipu_config = IPUConfig.from_pretrained(
    ipu_config_name,
    executable_cache_dir = executable_cache_dir,
    gradient_accumulation_steps=gradient_accumulation_steps,
    device_iterations = 32,
    replication_factor=1,
    inference_device_iterations = 64,
    inference_replication_factor = 1
)

The IPUTrainingArguments define any custom parameter modification we want to do, such as the initial learning rate for the model. It also allows other options, such as dataloader paramters, micro batch sizes and an automatic push to the Huggingface Hub (if credentials were set up earlier) to happen at given intervals.

These arguments are passed to the `IPUTrainer` which wraps the model training and evaluation process into a simple single-line process, doing all of the heavy lifting for us regarding training and evaluation loops, device assignment, optimiser definition, dataloading etc.

Note that only some arbitrary hyperparameter tuning was performed for this task. Other tasks and datasets may require further tuning to get the most optimal results.

In [ ]:
metric_name = "roc_auc"

args = IPUTrainingArguments(
    "./"+f"{model_name}-{model_task}",
    per_device_train_batch_size=micro_batch_size,
    per_device_eval_batch_size=4,
    num_train_epochs=5,
    learning_rate=2e-4,
    adam_epsilon=1e-6,
    loss_scaling=16.0,
    warmup_ratio=0.1,
    weight_decay=0,
    lr_scheduler_type = "cosine",
    metric_for_best_model=metric_name,
    dataloader_drop_last=True,
    dataloader_mode="async_rebatched",
    logging_steps=1,
    pod_type=pod_type,
    gradient_accumulation_steps=gradient_accumulation_steps,
    push_to_hub=True    
)

trainer = IPUTrainer(
    model,
    ipu_config,
    args,
    train_dataset=packed_train_dataset,
    eval_dataset=packed_val_dataset,
    compute_metrics=compute_metrics
)

Then, to train the model we can simply call the `train()` method:

In [ ]:
trainer.train()

***About the performance:*** `IPUTrainer` doesn't take into account that we have packed data samples when computing the speed metrics. It treats a 'sample' as a single input to the model, i.e. one **pack**.

So the actual throughput estimation can be obtained by multiplying the `samples_per_second` by the average packing factor (the average number of samples per pack) of the dataset. These were obtained in the `packing_algorithm` section: `5.68` for the `go-emotions` training set and `5.83` for validation set.


Next, we can evaluate the model by simply calling the `evaluate()` method:

In [ ]:
trainer.evaluate()

You can now upload the result of the training to the Hub if you successfully logged in at the beginning of this notebook, just execute this instruction:

In [ ]:
trainer.push_to_hub()

You can also save the model locally:

In [ ]:
trainer.save_model("./"+f"{model_name}-{model_task}")

You have now successfully fine-tuned and evaluated your speed-optimised model for text classification using packing!

## Faster inference

This section demonstrates how to perform faster, batched inference with a large number of samples using packing.

When training, the packing factor affects the convergence and hyperparameters in a similar way to a large increase in batch size. However, for inference-only runs, we are free to use a bigger packing factor to speed it up. Let's try it on GoEmotions with `max_seq_per_pack = 12`.

In [ ]:
max_seq_per_pack = 12

In [ ]:
dataset = load_dataset("go_emotions")

encoded_dataset = dataset.map(id_to_N_hot)
encoded_dataset = encoded_dataset.map(preprocess_function, batched=True)
inference_dataset = encoded_dataset['train'] #Lets use the train dataset to have more features

# The dataset initialisation and .create() can be executed in one line
inference_packed_dataset = PackedDatasetCreator(
    tokenized_dataset = encoded_dataset['train'],
    max_sequence_length = max_seq_length,
    max_sequences_per_pack = max_seq_per_pack,
    inference = True,
    problem_type = problem_type).create()



We can see that the average packing factor has improved from 5.68 to 9.14, allowing an approximate **9 times** throughput speed-up from the base unpacked model.

Let's also modify the configuration of the model for inference. For speed up, we can replicate a one-IPU run (`ipus_per_replica`) over four IPUs by changing the `replication_factor`. After this, we can re-initialise the model and the `IPUTrainer` with the existing arguments.

In [ ]:
ipu_config.layers_per_ipu = [12]
ipu_config.inference_device_iterations = 32
ipu_config.inference_replication_factor = 4
ipu_config.ipus_per_replica = 1

Let's load the checkpoint we saved earlier to run the inference on:

In [ ]:
model_checkpoint = f"./{model_name}_{model_task}"

# Load from Huggingface Hub instead:
# model_checkpoint = '<your_username>/{model_name}-{model_task}'

model = PackedPipelinedBertForSequenceClassification.from_pretrained(model_checkpoint, config=config)

In [ ]:
args = IPUTrainingArguments(
    "/tmp/"+f"{model_name}-{model_task}-fast-inf",
    per_device_eval_batch_size=8,
    dataloader_mode="async_rebatched",
    dataloader_drop_last=True,
    logging_steps=10,
    pod_type=pod_type
)

trainer = IPUTrainer(
    model,
    ipu_config,
    args,
    eval_dataset=inference_packed_dataset,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.evaluate(inference_packed_dataset)

Using these simple optimisations and the increase in maximum sequences per pack, we can see a massive throughput increase to approximately **25k sequences per second** - remember that to obtain the actual throughput we multiply the packed samples/s by the average packing factor - highlighting the benefits of using packing! 